In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path

import uproot
import numpy as np
import pandas as pd
import awkward as ak
import matplotlib.pyplot as plt

In [ ]:
here_path = Path.cwd()
data_path = here_path / ".." / "acts" / "ckf-missing-measurements" / "output"

assert(here_path.exists())

In [ ]:
particles = uproot.open(data_path / "particles_simulation.root")
particles = ak.to_dataframe(particles["particles"].arrays(library="ak"), how="outer")

particles

In [ ]:
hits = uproot.open(data_path / "hits.root")
hits = ak.to_dataframe(hits["hits"].arrays(library="ak"), how="outer")

hits

In [ ]:
measurements = uproot.open(data_path / "measurements.root")

measurements = pd.concat(
    ak.to_dataframe(measurements[f"vol{vol}"].arrays(
        ["event_nr", "volume_id", "layer_id", "surface_id", "true_loc0", "true_loc1", "rec_loc0"] + (["rec_loc1"] if vol < 28 else []),
        library="ak",
    ), how="outer")
    for vol in [16, 17, 18, 23, 24, 25, 28, 29, 30]
)

measurements

In [ ]:
tracks = uproot.open(data_path / "tracksummary_ambi.root")
tracks = ak.to_dataframe(tracks["tracksummary"].arrays(["event_nr", "track_nr", "nMeasurements", "nHoles", "nOutliers"], library="ak"), how="outer")

tracks

In [ ]:
trackstates = uproot.open(data_path / "trackstates_ambi.root")
trackstates = ak.to_dataframe(trackstates["trackstates"].arrays(["event_nr", "volume_id", "layer_id", "module_id", "stateType", "t_x", "t_y", "t_z", "g_x_flt", "g_y_flt", "g_z_flt", "eLOC0_prt", "eLOC1_prt", "res_eLOC0_prt", "res_eLOC1_prt", "res_ePHI_prt", "res_eTHETA_prt", "res_eLOC0_smt", "res_eLOC1_smt", "res_ePHI_smt", "res_eTHETA_smt"], library="ak"), how="outer")

trackstates

In [ ]:
track_particles = pd.merge(
    left=tracks.add_prefix("track_"),
    right=particles.add_prefix("particle_"),
    left_on="track_event_nr",
    right_on="particle_event_id",
)

track_particles

In [ ]:
track_particles["abs_hit_difference"] = track_particles["particle_number_of_hits"] - track_particles["track_nMeasurements"]

tmp = track_particles[track_particles["abs_hit_difference"] != 0].sort_values(by="abs_hit_difference", ascending=False)

print(tmp["track_nHoles"].sum())
print(tmp["track_nOutliers"].sum())

tmp

In [ ]:
event_nr = 3155
background = True

fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.set_xlabel("z")
ax.set_ylabel("x")
ax.set_zlabel("y")

my_hits = hits[hits["event_id"] == event_nr]
my_track = tracks[tracks["event_nr"] == event_nr]
my_trackstates = trackstates[trackstates["event_nr"] == event_nr]
my_trackstates = my_trackstates[my_trackstates["stateType"] == 0]

print(my_hits[["volume_id", "layer_id", "sensitive_id"]])
print(my_trackstates[["volume_id", "layer_id", "module_id", "stateType"]])

ax.scatter(my_hits["tz"], my_hits["tx"], my_hits["ty"], alpha=1.0, marker="o", s=50)
ax.scatter(my_trackstates["g_z_flt"], my_trackstates["g_x_flt"], my_trackstates["g_y_flt"], alpha=1.0, marker="^", s=100)

if background:
    random_hits = hits.sample(10000)
    ax.scatter(random_hits["tz"], random_hits["tx"], random_hits["ty"], c="grey", alpha=0.3, marker=".", s=1)

plt.show()

In [ ]:
volume = 24
layer = 4

my_measurements = measurements
my_measurements = my_measurements[my_measurements["volume_id"] == volume]
my_measurements = my_measurements[my_measurements["layer_id"] == layer] if layer is not None else my_measurements

measurements_tracks = pd.merge(
    left=tracks.dropna(),
    right=my_measurements,
    left_on=["event_nr"],
    right_on=["event_nr"],
    how="inner",
)

measurements_trackstates = pd.merge(
    left=trackstates[trackstates["stateType"] == 0],
    right=measurements_tracks,
    left_on=["event_nr", "volume_id", "layer_id", "module_id"],
    right_on=["event_nr", "volume_id", "layer_id", "surface_id"],
    how="outer",
)

fig = plt.figure(figsize=(10, 4))
axs = fig.subplots(1,2)

measurement_without_trackstate_mask = measurements_trackstates["module_id"].isna()
measurement_without_trackstate = measurements_trackstates[measurement_without_trackstate_mask]

axs[0].scatter(my_measurements["true_loc0"], my_measurements["true_loc1"], alpha=0.01, label="hits")
axs[0].scatter(measurement_without_trackstate["true_loc0"], measurement_without_trackstate["true_loc1"], alpha=0.5, label="not matched hits")

axs[1].scatter(my_measurements["rec_loc0"], my_measurements["rec_loc1"], alpha=0.01, label="measurements")
axs[1].scatter(measurement_without_trackstate["rec_loc0"], measurement_without_trackstate["rec_loc1"], alpha=0.5, label="not matched measurements")

fig.suptitle(f"vol={volume} layer={layer}")
axs[0].set_title("hits")
axs[1].set_title("measurements")
for ax in axs:
    ax.set_xlabel("local x")
    ax.set_ylabel("local y")
    ax.legend()

print(f"measurements {len(my_measurements)} without trackstate {len(measurement_without_trackstate)}")

plt.show()

In [ ]:
event_nr = 3735
volume = 24
layer = 2
sensor = 693

my_measurements = measurements
my_measurements = my_measurements[my_measurements["event_nr"] == event_nr]
my_measurements = my_measurements[my_measurements["volume_id"] == volume]
my_measurements = my_measurements[my_measurements["layer_id"] == layer] if layer is not None else my_measurements
my_measurements = my_measurements[my_measurements["surface_id"] == sensor] if sensor is not None else my_measurements
#print(my_measurements)
#print(my_measurements[["true_loc0", "true_loc1"]].values - my_measurements[["rec_loc0", "rec_loc1"]].values)

my_trackstates = trackstates
my_trackstates = my_trackstates[my_trackstates["event_nr"] == event_nr]
my_trackstates = my_trackstates[my_trackstates["volume_id"] == volume]
my_trackstates = my_trackstates[my_trackstates["layer_id"] == layer] if layer is not None else my_measurements
my_trackstates = my_trackstates[my_trackstates["module_id"] == sensor] if sensor is not None else my_measurements
#print(my_trackstates)

fig = plt.figure()
ax = fig.add_subplot()

ax.set_title(f"vol={volume} layer={layer}")
ax.set_xlabel("local x")
ax.set_ylabel("local y")

ax.scatter(my_measurements["true_loc0"], my_measurements["true_loc1"], label="true")
ax.scatter(my_measurements["rec_loc0"], my_measurements["rec_loc1"], label="cluster")
ax.scatter(my_trackstates["eLOC0_prt"], my_trackstates["eLOC1_prt"], label="track")

ax.legend()

plt.show()